<a href="https://colab.research.google.com/github/Singh07-Shubham/Custom-Object-Detection/blob/master/kanbanwithtextblock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 12886, done.
remote: Total 12886 (delta 0), reused 0 (delta 0), pack-reused 12886
Receiving objects: 100% (12886/12886), 11.72 MiB | 11.43 MiB/s, done.
Resolving deltas: 100% (8786/8786), done.


In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [4]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:909:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/

In [5]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'		      images	       SKG
 CSS_Training			      Images	       SpringEMSProject
 demo.mp4			      ponzi.txt        yolov3
'EOD Updates - DataScience .gsheet'   Resume.gdoc      Youtube_Share
'fxcmtemp (1).sql'		      Resume.pdf
 html_css_bootstrap_test	      Sharing_Caring


In [0]:
# copy the .zip file into the root directory of cloud VM
!cp /mydrive/yolov3/kanban_new/obj.zip ../

In [9]:
# unzip the zip file and its contents should now be in /darknet/data/obj
!unzip ../obj.zip -d darknet/data/

Archive:  ../obj.zip
   creating: darknet/data/obj/
  inflating: darknet/data/obj/images300.txt  
  inflating: darknet/data/obj/image0.jpeg  
  inflating: darknet/data/obj/image8.jpeg  
  inflating: darknet/data/obj/images224.txt  
  inflating: darknet/data/obj/47d255dbf132a9e7033f7fade840c48f--scrum-board-diy-kanban-board.txt  
  inflating: darknet/data/obj/255abfb3d53f59e9229d8721e55f50c8.txt  
  inflating: darknet/data/obj/4211-editable-kanban-board-powerpoint-templates-16x9-2.txt  
  inflating: darknet/data/obj/4211-editable-kanban-board-powerpoint-templates-16x9-3.txt  
  inflating: darknet/data/obj/4211-editable-kanban-board-powerpoint-templates-16x9-4.txt  
  inflating: darknet/data/obj/Agile-post-it-notes.txt  
  inflating: darknet/data/obj/b409a555-636e-4d1b-a608-f26ac7a87ba2.txt  
  inflating: darknet/data/obj/images9.jpg  
  inflating: darknet/data/obj/images10.jpg  
  inflating: darknet/data/obj/images12.jpg  
  inflating: darknet/data/obj/image2.txt  
  inflating: darknet/

In [10]:
cd darknet

/content/darknet


In [0]:
!cp /mydrive/yolov3/kanban_new/yolov3_custom.cfg cfg/

In [0]:
!cp /mydrive/yolov3/kanban_new/obj.names data/
!cp /mydrive/yolov3/kanban_new/obj.data data/

In [0]:
# upload the generate_train.py script to cloud VM from Google Drive
!cp /mydrive/yolov3/kanban_new/generate_train.py ./

In [0]:
!python generate_train.py

In [15]:
!ls data/

9k.tree     eagle.jpg	 imagenet.labels.list	   obj.data	     scream.jpg
coco9k.map  giraffe.jpg  imagenet.shortnames.list  obj.names	     train.txt
coco.names  goal.txt	 labels			   openimages.names  voc.names
dog.jpg     horses.jpg	 obj			   person.jpg


In [16]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

--2020-03-24 04:17:44--  http://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/darknet53.conv.74 [following]
--2020-03-24 04:17:44--  https://pjreddie.com/media/files/darknet53.conv.74
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.30MB/s    in 1m 44s  

2020-03-24 04:19:29 (1.48 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
!cp /mydrive/yolov3/kanban_new/obj.data data/

In [0]:
# train your custom detector
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.239196, GIOU: 0.141017), Class: 0.583449, Obj: 0.470915, No Obj: 0.489278, .5R: 0.000000, .75R: 0.000000, count: 9, class_loss = 222.087189, iou_loss = 7.670776, total_loss = 229.757965 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.297379, GIOU: 0.138586), Class: 0.491113, Obj: 0.495850, No Obj: 0.479889, .5R: 0.114754, .75R: 0.000000, count: 61, class_loss = 858.281433, iou_loss = 61.295654, total_loss = 919.577087 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.317998, GIOU: 0.178823), Class: 0.576823, Obj: 0.532278, No Obj: 0.549278, .5R: 0.090909, .75R: 0.000000, count: 11, class_loss = 4409.064453, iou_loss = 10.239258, total_loss = 4419.303711 

 14: 1841.515259, 2133.983154 avg loss, 0.000000 rate, 10.017452 seconds, 896 images, 12.772525 time left
Loaded: 0.000055 seconds
v3 (mse loss, Normalizer: (iou: 0.75